In [2]:
from euroleague_api.play_by_play_data import PlayByPlay 
from euroleague_api.game_stats import GameStats
from euroleague_api.shot_data import ShotData
import warnings
import pandas as pd
import statsmodels.api as sm
import numpy as np
# Masquer tous les warnings
warnings.filterwarnings("ignore")
import random

In [95]:
competition ="euroleague"
#season = 2023

data_reg = pd.DataFrame(columns=[])

if competition == "euroleague" :
    pbp_ = PlayByPlay(competition="E")
    gs_ = GameStats(competition="E")
    sd_ = ShotData(competition="E")

else :
    pbp_ = PlayByPlay(competition="U")
    gs_ = GameStats(competition="U")
    sd_ = ShotData(competition="U")

for season in [2021,2022,2023] :
    for gc in range(306) : 
        gamecode=gc + 1

        try :
                
            gs = gs_.get_game_report(season=season, game_code=gamecode)
            pbp = pbp_.get_game_play_by_play_data(season=season, gamecode=gamecode)
            #sd = sd_.get_game_shot_data(season=season,gamecode=gamecode)
            HOME_TEAM = gs["local.club.code"].tolist()[0]
            AWAY_TEAM = gs["road.club.code"].tolist()[0]
            pbp['PLAYTYPE'] = pbp['PLAYTYPE'].str.replace(' ', '', regex=True)
            keep_type = ["D","2FGM","RV","CM","AS","2FGA","3FGA","FTM","TO","O","3FGM","ST","FTA","FV","AG","OF","CMU","CMT","CMD","CMTI"]
            pbp = pbp[pbp['PLAYTYPE'].isin(keep_type)].reset_index(drop=True)

            home_score = pbp["POINTS_A"].max()
            away_score = pbp["POINTS_B"].max()
            pbp_home = pbp[(pbp["CODETEAM"] == HOME_TEAM)&(pbp["PLAYER_ID"]!="")]
            pbp_away = pbp[(pbp["CODETEAM"] == AWAY_TEAM)&(pbp["PLAYER_ID"]!="")]

            # Utilisation de value_counts pour compter les occurrences
            value_counts = pbp_home['PLAYTYPE'].value_counts()
            # Transformation en DataFrame à une ligne
            result_home = pd.DataFrame([value_counts]).reset_index(drop=True)
            result_home["Y"] = home_score - away_score
            data_reg = pd.concat([data_reg,result_home]).reset_index(drop=True)

            # Utilisation de value_counts pour compter les occurrences
            value_counts = pbp_away['PLAYTYPE'].value_counts()
            # Transformation en DataFrame à une ligne
            result_away = pd.DataFrame([value_counts]).reset_index(drop=True)
            result_away["Y"] = away_score - home_score
            data_reg = pd.concat([data_reg,result_away]).reset_index(drop=True)
        except :
            pass

In [120]:
data_reg = data_reg.fillna(0)

In [122]:
data_reg["CMU"] = data_reg["CMU"] + data_reg["CMD"] + data_reg["CMTI"] + data_reg["CMT"]
data_reg["TO"] = data_reg["TO"] + data_reg["OF"] 

In [ ]:
"CMD" -> "CMU"
"CMTI" -> "CMU"
"CMT" -> "CMU"
"OF" -> "TO"

In [133]:
# Définir les variables explicatives X (toutes sauf celles spécifiées)
X = data_reg.drop(columns=['Y',"CMD","CMTI","CMT","OF","AG"])


# Définir la variable cible Y
Y = data_reg['Y']

# Ajustement du modèle de régression linéaire pondérée
model = sm.OLS(Y, X)
results = model.fit()


# Création du tableau des coefficients
coeffs = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': results.params,
})
# Trier le tableau des coefficients du plus grand au plus petit
coeffs_sorted = coeffs.sort_values(by='Variable', ascending=True).reset_index(drop=True)

In [174]:
pbp2024 = pd.read_csv("euroleague_pbp_2024.csv")

coefficients = {
    'D': 0.8,
    '2FGA': -1,
    '2FGM': 1.5,
    'AS': 0.5,
    'RV': 0.5,
    'CM': -0.5,
    '3FGA': -0.6,
    'FTM': 0.75,
    '3FGM': 2.25,
    'O': 1.2,
    'TO': -1.5,
    'ST': 1.5,
    'FTA': -0.8,
    'FV': 0.5,
    'CMU': -1.5,
    'CMD': -1.5,
    'CMTI': -1.5,
    'CMT': -1.5,
    'OF': -1.5
}

# Création de la nouvelle colonne 'Nouveau_Coeff' en utilisant map et en attribuant 0 par défaut si non trouvé
pbp2024['EVAL'] = pbp2024['PLAYTYPE'].map(coefficients).fillna(0)
pbp2024_play = pbp2024[~pbp2024["PLAYER"].isna()]
# Calcul de la somme de EVAL pour chaque joueur
somme_eval_par_joueur = pbp2024_play.groupby(['CODETEAM','PLAYER','PLAYER_ID'])['EVAL'].sum().reset_index()

# Renommage des colonnes
somme_eval_par_joueur.columns = ['CODETEAM','PLAYER','PLAYER_ID', 'PER_V0']

somme_eval_par_joueur = somme_eval_par_joueur.sort_values(by = "PER_V0", ascending=False).reset_index(drop=True)
somme_eval_par_joueur.insert(0,"RANK",range(1,len(somme_eval_par_joueur)+1))

In [176]:
somme_eval_par_joueur.reset_index(drop=True)

,RANK,CODETEAM,PLAYER,PLAYER_ID,PER_V0
0,1,ASV,"SAKO, NEAL",P010545,82.85
1,2,OLY,"VEZENKOV, SASHA",P003469,66.60
2,3,ASV,"MALEDON, THEO",P007027,66.20
3,4,PAN,"LESSORT, MATHIAS",P003842,64.45
4,5,BAS,"MONEKE, CHIMA",P012201,59.20
...,...,...,...,...,...
225,226,ZAL,"LEKAVICIUS, LUKAS",P003210,-3.85
226,227,ZAL,"WALLACE, TYRONE",P011925,-4.30
227,228,PRS,"OUATTARA, YAKUBA",P004197,-5.30
228,229,ASV,"KAHUDI, CHARLES",PKIR,-7.10


In [154]:
# Affichage du DataFrame résultant
somme_eval_par_joueur.head(50)

,TEAM,PLAYER,Somme_EVAL
0,ASV,"SAKO, NEAL",95.6
1,ASV,"MALEDON, THEO",83.7
2,OLY,"VEZENKOV, SASHA",83.1
3,PAN,"LESSORT, MATHIAS",80.2
4,BAR,"PUNTER, KEVIN",75.5
5,BAS,"MONEKE, CHIMA",75.2
6,MUN,"NAPIER, SHABAZZ",67.8
7,PRS,"SHORTS, TJ",67.2
8,PAN,"NUNN, KENDRICK",65.7
9,MCO,"OKOBO, ELIE",65.6


                    PLAYER  Somme_EVAL
0         LESSORT, MATHIAS       696.1
1              JAMES, MIKE       661.9
2               NEBO, JOSH       636.0
3            LARKIN, SHANE       608.1
4           COLSON, BONZIE       604.1
5            MONEKE, CHIMA       583.6
6             PETERS, ALEC       568.7
7       HAYES-DAVIS, NIGEL       563.4
8            EVANS, KEENAN       554.9
9           HEZONJA, MARIO       532.1
10             VESELY, JAN       532.0
11  MITOGLOU, KONSTANTINOS       522.6
12            MUSA, DZANAN       497.6
13      SEDEKERSKIS, TADAS       496.6
14       WILBEKIN, SCOTTIE       491.7
15      SHENGELIA, TORNIKE       485.9
16       CAMPAZZO, FACUNDO       485.0
17          HOWARD, MARKUS       476.5
18          BOLOMBOY, JOEL       468.8
19        POIRIER, VINCENT       466.1
20         SHIELDS, SHAVON       459.0
21         SLOUKAS, KOSTAS       453.3
22   MILLER-MCINTYRE, CODI       449.2
23           DIALLO, ALPHA       447.9
24        BALDWIN IV, WAD

In [126]:

# Calcul de la moyenne de chaque colonne
moyenne_colonnes = data_reg.mean()

# Conversion des moyennes en DataFrame
moyenne_df = pd.DataFrame(moyenne_colonnes).reset_index()
moyenne_df.columns = ['Variable', 'Moyenne']  # Renommer les colonnes

# Tri du DataFrame par moyenne croissante
moyenne_df_sorted = moyenne_df.sort_values(by='Variable')
moyenne_df_sorted = moyenne_df_sorted[~moyenne_df_sorted["Variable"].isin(['Y',"CMD","CMTI","CMT","OF"])].reset_index(drop=True)

In [127]:
merged_df = coeffs_sorted.merge(moyenne_df_sorted, left_on='Variable', right_on='Variable', how='inner')
# merged_df['Coefficient2'] = (merged_df["Coefficient"] + 0.5).round(1)
# merged_df['TOT'] = merged_df["Coefficient2"] * merged_df["Moyenne"]

In [134]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      Y   R-squared (uncentered):                   0.669
Model:                            OLS   Adj. R-squared (uncentered):              0.666
Method:                 Least Squares   F-statistic:                              237.7
Date:                Sun, 20 Oct 2024   Prob (F-statistic):                        0.00
Time:                        00:33:13   Log-Likelihood:                         -6128.5
No. Observations:                1780   AIC:                                  1.229e+04
Df Residuals:                    1765   BIC:                                  1.237e+04
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
D              1.3485      0.045     29.677      0.000       1.259       1.438
2FGA          -1.5028      0.047    -31.915      0.000      -1.595      -1.410
2FGM           0.2970      0.050      5.889      0.000       0.198       0.396
AS             0.0110      0.053      0.206      0.837      -0.094       0.116
RV             0.1204      0.079      1.532      0.126      -0.034       0.274
CM            -0.2610      0.057     -4.595      0.000      -0.372      -0.150
3FGA          -1.5511      0.049    -31.669      0.000      -1.647      -1.455
FTM            0.1207      0.051      2.356      0.019       0.020       0.221
3FGM           1.1327      0.074     15.224      0.000       0.987       1.279
O              1.4778      0.071     20.929      0.000       1.339       1.616
TO            -1.2112      0.044    -27.561      0.000      -1.297      -1.125
ST             1.6031      0.073     21.907      0.000       1.460       1.747
FTA           -0.5070      0.089     -5.684      0.000      -0.682      -0.332
FV             0.2568      0.110      2.327      0.020       0.040       0.473
CMU           -1.2680      0.266     -4.764      0.000      -1.790      -0.746
==============================================================================
Omnibus:                       10.679   Durbin-Watson:                   2.274
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               12.249
Skew:                          -0.118   Prob(JB):                      0.00219
Kurtosis:                       3.331   Cond. No.                         80.7
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
match = [
    [1,2],
    [1,3],
    [1,4],
    [1,5],
    [2,3],
    [2,4],
    [2,5],
    [3,4],
    [3,5],
    [4,5]
    ]

#faire un df à 10 colonnes, M1 à M10, chaque ligne coresponds à une composition unique d'ordre des 10 matchs.

In [40]:
# Re-importation des bibliothèques nécessaires après le reset de l'environnement
import pandas as pd
import itertools

# Liste des matchs
matches = [
    [1, 2],
    [1, 3],
    [1, 4],
    [1, 5],
    [2, 3],
    [2, 4],
    [2, 5],
    [3, 4],
    [3, 5],
    [4, 5]
]

# Filtrer les permutations pour ne conserver que celles avec [2,3] en M1 et [4,5] en M10
filtered_permutations = [p for p in itertools.permutations(matches) if 1 in p[0] and 1 in p[-2] and p[-1] == [4, 5] and p[-2] != [1,2]]

# Créer le DataFrame à partir de ces permutations filtrées
filtered_df = pd.DataFrame(filtered_permutations, columns=[f'M{i+1}' for i in range(10)])


In [41]:
filtered_df["M1_5"] = filtered_df[[f"M{i+1}" for i in range(5)]].apply(
    lambda row: [item for match in row for item in match], axis=1
)

In [42]:
def has_exactly_two_each(lst):
    counts = {i: lst.count(i) for i in range(1, 6)}
    return all(counts[i] == 2 for i in range(1, 6))

# Appliquer le filtre
filtered_exact_df = filtered_df[filtered_df["M1_5"].apply(has_exactly_two_each)]
filtered_exact_df = filtered_exact_df.drop("M1_5",axis = 1).reset_index(drop = True)

In [44]:
def has_unique_four_across_pairs(row):
    for i in range(9):  # De M1 à M9
        combined = row[f"M{i+1}"] + row[f"M{i+2}"]
        if len(set(combined)) != 4:
            return False
    return True

# Appliquer le filtre
final_filtered_df = filtered_exact_df[filtered_exact_df.apply(has_unique_four_across_pairs, axis=1)].reset_index(drop = True)


In [45]:
final_filtered_df

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10
0,"[1, 4]","[2, 3]","[1, 5]","[2, 4]","[3, 5]","[1, 2]","[3, 4]","[2, 5]","[1, 3]","[4, 5]"
1,"[1, 5]","[2, 3]","[1, 4]","[2, 5]","[3, 4]","[1, 2]","[3, 5]","[2, 4]","[1, 3]","[4, 5]"


In [28]:
final_filtered_df["M1"].value_counts()

M1
[1, 4]    4
[1, 5]    4
[2, 4]    4
[2, 5]    4
[3, 4]    4
[3, 5]    4
Name: count, dtype: int64